In [ ]:
#Importing all the required packages
import torch as tr
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import math
import time
import sys
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
#Defining some of the functions used
def de2bi(d, n):
	d = np.array(d)
	power = 2**np.arange(n)
	return (np.floor((d[:,None]%(2*power))/power))

def bi2de(L,m):
    Ldec = np.zeros((int(2**m),1), dtype = np.int)
    for i in range(int(m)):
        for j in range(int(2**m)):
            Ldec[j] = Ldec[j] + L[j,i]*2**i 
    return Ldec
    
def get_labeling(m):
    M  = 2**m
    if m == 1:
        L = np.asarray([[0],[1]])
    else:
        L = np.zeros((M,m))
        L[0:int(M/2),1:m] = get_labeling(m-1)
        L[int(M/2):M, 1:m] = np.flipud(L[0:int(M/2),1:m])
        L[int(M/2):M,0] = 1
    return L

def get_constellation(M):
    Delta = np.sqrt(3/(2*(M-1)))
    Xpam = np.expand_dims(Delta*np.arange(-(np.sqrt(M)-1),np.sqrt(M)+1,2),axis = 1)
    xpamt_2D = np.tile(Xpam,(1,int(np.sqrt(M))))
    xpamt = np.expand_dims(xpamt_2D.flatten(),axis = 1)
    X = np.transpose(np.reshape(np.asarray([xpamt, np.tile(Xpam,(int(np.sqrt(M)),1))]),(2,M)))
    Ltmp = get_labeling(int(np.log2(M)/2))
    Ltmp_dec = bi2de(Ltmp,int(np.log2(M))/2)
    Ltmp_dec2 = np.tile(Ltmp_dec,(1,int(np.sqrt(M))))
    Ltmp_dec3 = np.expand_dims(Ltmp_dec2.flatten(),axis = 1)
    L = np.concatenate((np.reshape(de2bi(Ltmp_dec3,int(np.log2(M)/2)),(M,int(np.log2(M)/2))), np.tile(Ltmp,(int(np.sqrt(M)),1))), axis = 1)
    Ldec = np.reshape(np.asarray(bi2de(np.fliplr(L),int(np.log2(M))),dtype = np.int),M)
    return [X,Ldec]

def get_APSK(M2,Amount_of_Rings):
    M = int(M2/Amount_of_Rings) * np.ones(Amount_of_Rings, dtype = np.int)
    X = np.zeros((sum(M),2))
    if Amount_of_Rings > 1:
        idx = 0
        l_r1 = get_labeling(int(np.log2(M[0])))*M.shape[0]
        ldec_r1 = bi2de(l_r1,int(np.log2(M[0])))
        print('a')
        l_rs = get_labeling(int(np.log2(M.shape[0])))
        ldec_rs = bi2de(l_rs,int(np.log2(M.shape[0])))
        l_apsk = np.zeros((sum(M),1),dtype = np.int)
        for i in range(M.shape[0]):
            R = np.sqrt(-np.log(1-(i+0.5)*1/M.shape[0]))
            for j in range(M[i]):
                X[idx+j,:] = [R*np.cos(j*2*math.pi/M[i]), R*np.sin(j*2*math.pi/M[i]) ]
            l_apsk[idx:idx+M[i],] = ldec_r1 + ldec_rs[i]
            idx = idx + M[i]
        l_apsk = np.squeeze(l_apsk)
    else:
        Lbin = get_labeling(int(np.log2(M[0])))
        l_apsk = np.squeeze(bi2de(Lbin,int(np.log2(M[0]))))
        for j in range(M2):
             X[j,:] = [np.cos(j*2*math.pi/M2), np.sin(j*2*math.pi/M2)]
    return [X, l_apsk]

def get_constellation_4D(M):
    m = int(np.log2(M))
    m_pd = int(m/4)
    Delta = np.sqrt(np.sqrt(3/(2*(M-1))))
    Xpam = np.expand_dims(Delta*np.arange(-(np.sqrt(np.sqrt(M))-1),np.sqrt(np.sqrt(M))+1,2),axis = 1)
    Lpam = get_labeling(m_pd)
    X1 =  (np.expand_dims(np.ones(int(M/2**m_pd)),axis= 1)*np.transpose(Xpam)).flatten('F')
    X2 =  np.tile((np.expand_dims(np.ones(int(M/2**(m_pd*2))),axis= 1)*np.transpose(Xpam)).flatten('F'), 2**m_pd)
    X3 =  np.tile((np.expand_dims(np.ones(int(M/2**(m_pd*3))),axis= 1)*np.transpose(Xpam)).flatten('F'), 2**(m_pd*2))
    X4 =  np.tile((np.expand_dims(np.ones(int(M/2**(m_pd*4))),axis= 1)*np.transpose(Xpam)).flatten('F'), 2**(m_pd*3))
    L1 =  np.reshape(np.tile(Lpam,(int(M/2**m_pd))), (M,m_pd))
    L2 =  np.tile(np.reshape(np.tile(Lpam,(int(M/2**(m_pd*2)))), (int(M/2**m_pd),m_pd)),(2**m_pd,1))
    L3 =  np.tile(np.reshape(np.tile(Lpam,(int(M/2**(m_pd*3)))), (int(M/2**(m_pd*2)),m_pd)),(2**(m_pd*2),1))
    L4 =  np.tile(np.reshape(np.tile(Lpam,(int(M/2**(m_pd*4)))), (int(M/2**(m_pd*3)),m_pd)),(2**(m_pd*3),1))
    X = np.transpose(np.asarray([X1,X2,X3,X4]))
    Lbin =np.asarray(np.concatenate((L1,L2,L3,L4),axis = 1), dtype = np.int)
    Ldec = np.squeeze(bi2de(Lbin,m))
    return [X,Ldec]  

def get_APSK_4D(M, Amount_of_Rings):
    m = int(np.log2(M))
    M2 = int(np.sqrt(M))
    X_2D, l_apsk_2D = get_APSK(M2,Amount_of_Rings)
    Lbin_2D = de2bi(l_apsk_2D, int(m/2))
    X1 = np.repeat(X_2D,M2,axis = 0)
    X2 = np.reshape(np.repeat(X_2D,M2,axis = 1), (M,2), 'F')
    L1 = np.repeat(Lbin_2D, M2, axis = 0)
    L2 = np.reshape(np.repeat(Lbin_2D,M2,axis = 1), (M,int(m/2)), 'F')
    X = np.concatenate([X1,X2], axis = 1)
    Lbin = np.concatenate([L1,L2], axis = 1)
    Ldec = np.squeeze(bi2de(Lbin,m))
    return[X,Ldec]

def get_Optimized_4D(X_2D, l_2D, M):
    m = int(np.log2(M)) 
    M2 = int(np.sqrt(M))
    Lbin_2D = de2bi(l_2D, int(m/2))
    X1 = np.repeat(X_2D,M2,axis = 0)
    X2 = np.reshape(np.repeat(X_2D,M2,axis = 1), (M,2), 'F')
    L1 = np.repeat(Lbin_2D, M2, axis = 0)
    L2 = np.reshape(np.repeat(Lbin_2D,M2,axis = 1), (M,int(m/2)), 'F')
    X = np.concatenate([X1,X2], axis = 1)
    Lbin = np.concatenate([L1,L2], axis = 1)
    Ldec = np.squeeze(bi2de(Lbin,m))
    return[X,Ldec]

def get_Random_4D(M):
    m = int(np.log2(M))
    M2 = int(M/16)
    X_G = abs(np.random.normal(0,1,(M2,4)))
    L_2D = get_labeling(m)
    L_Q = get_labeling(4)
    X_T = np.tile(X_G,[16,1])
    Q_M = 2*L_Q -1
    Q_M = np.repeat(Q_M,M2, axis = 0)
    X_RN = X_T*Q_M
    L_2D = np.tile(L_2D,[16,1])
    L_Q = np.repeat(L_Q,M2,axis = 0)
    Lbin = np.concatenate([L_2D,L_Q],axis = 1)
    Ldec = np.squeeze(bi2de(Lbin,m))
    return[X_RN,Ldec]

def awgn_channel(x):
    return x + np.sqrt(sigma2)*tr.randn(M*stacks,channel_uses).to(Device)

def normalization(x): # E[|x|^2] = 1
    return x / tr.sqrt((channel_uses*(x**2)).mean())

def save():
        tr.save({
         'model_state_dict' : encoder.state_dict(), 
         'optimizer_state_dict': optimizer.state_dict(),
         'loss': loss_history,
         'constellations': Constellations,
         'SNR': EsNo_dB,
         'epochs' : epochs,
         'stacks' : stacks,
         'learning_rate': learning_rate,
         'Device': Device,
         'time': time.time()-start_time,
         'Ldec': idx_train,
         'Lbin': de2bi(idx_train,m)},'./Data/GMI/' + str(channel_uses) + 'D/' + str(M) + '/GMI_' + Estimation_type + '_' + str(channel_uses) + 'D_' + str(M) + '_' + str(EsNo_dB) + 'dB_'+ str(learning_rate)+'lr_' + Initial_Constellation)

        sio.savemat('./Data/GMI/' + str(channel_uses) + 'D/' + str(M) + '/GMI_' + Estimation_type + '_' + str(channel_uses) + 'D_' + str(M) + '_' + str(EsNo_dB) + 'dB_'+ str(learning_rate)+'lr_' + Initial_Constellation + '.mat', {
         'model_state_dict' : encoder.state_dict(), 
         'optimizer_state_dict': optimizer.state_dict(),
         'loss': loss_history,
         'constellations': Constellations,
         'SNR': EsNo_dB,
         'epochs' : epochs,
         'stacks' : stacks,
         'learning_rate': learning_rate,
         'Device': Device,
         'time': time.time()-start_time,
         'Ldec': idx_train,
         'Lbin': de2bi(idx_train,m)})


In [ ]:
M = 16 #The cardinality size of the constellation
stacks = 4 #The amount of noise samples per symbol for MC optimization
channel_uses = 4 #The dimensionality of the constellation
learning_rate = 0.1 #Learning Rate
EsNo_dB = 9 # The SNR
epochs = 800 #The amount of iterations
Estimation_type = 'GH' #GH or MC
Device = 'cpu' #Determines the device which the optimization is done on, 'cpu' for cpu and 'cuda:0', 'cuda:1' etc. for GPU
Initial_Constellation = 'APSK' #You can choose either 'APSK', 'QAM', and in the case of 4D 'Random' or the directory to an optimized 2D constellation
Amount_of_Rings  = 1 #Only relevant for APSK, tells you how many rings you want in the constellation, should be a power of 2

In [ ]:
#Defining some of the parameters of the optimization
m = int(np.log2(M)) #The amount of bits per symbol
EsNo_r = 10**(EsNo_dB/10)
sigma2 = 1/(channel_uses*EsNo_r) # noise variance per channel use
GH =  sio.loadmat('GaussHermite_J_10.mat')#Loading in the Gauss-Hermite points
X_eye = tr.eye(M).to(Device) # The input to our neural network
X_tilde = X_eye.repeat(stacks,1).to(Device)
if channel_uses == 2:
    if Initial_Constellation == 'QAM':
        [X_target,idx_train]= get_constellation(M)
    else:
        [X_target,idx_train] = get_APSK(M,Amount_of_Rings)
        Initial_Constellation = Initial_Constellation + '_' + str(Amount_of_Rings) + 'Rings'
else:
    if Initial_Constellation == 'QAM':
        [X_target,idx_train]= get_constellation_4D(M)
    elif Initial_Constellation == 'APSK':
        [X_target,idx_train] = get_APSK_4D(M,Amount_of_Rings)
        Initial_Constellation = Initial_Constellation + '_' + str(Amount_of_Rings) + 'Rings'
    elif Initial_Constellation == 'Random':
        [X_target,idx_train] = get_Random_4D(M)
    else:
        Optimized_Constellation_2D = tr.load(Initial_Constellation)
        X_2D = Optimized_Constellation_2D['constellations'][:,:,-1]
        l_2D =  Optimized_Constellation_2D['Ldec']
        [X_target,idx_train] = get_Optimized_4D(X_2D,l_2D,M)
        Initial_Constellation = 'Optimized_2D'
labeling = de2bi(np.arange(M), m)
X_target = tr.tensor(X_target,dtype = tr.float32).to(Device)
X_target = normalization(X_target)

In [ ]:
#=====================================================#
# build the computation graph
#=====================================================#
def GMI(X_tilde, idx):
    X = normalization(encoder(X_tilde))# minibatch_size x channel_uses
    idx2 = np.zeros(M)
    for i in range(M):
        idx2[idx[i]] = i
    X_sorted = tr.zeros((M*stacks,channel_uses)).to(Device)
    X_temp = X[idx2,:]
    for i in range(stacks):
        X_sorted[i*M:(i+1)*M,:] = X_temp[:,:]
    Y = awgn_channel(X_sorted)                 # minibatch_size x channel_uses
    # compute posterior distribution
    XX0_tmp,XX1_tmp,XX0,XX1 = [],[],[],[]
    for i in range(int(m)):
        XX0_tmp.clear()
        XX1_tmp.clear()
        for j in range(M):
            if labeling[j,i] == 0:
                XX0_tmp.append(tr.reshape(X_sorted[j,:], shape=[1,1,channel_uses]))
            else:
                XX1_tmp.append(tr.reshape(X_sorted[j,:], shape=[1,1,channel_uses]))
        XX0.append(tr.cat(XX0_tmp, 0))
        XX1.append(tr.cat(XX1_tmp, 0))
    XB = []
    for i in range(int(m)):
        tmp = tr.cat([XX0[i].repeat([1, int(2**i), 1]), XX1[i].repeat([1, int(2**i), 1])], 1)
        XB.append(tmp.repeat([1, int(2**(m-i-1)*stacks), 1]))
    XX = tr.reshape(X_sorted[0:M:,:], shape=[M, 1, channel_uses])
    YY = tr.reshape(Y, shape=[1, M*stacks, channel_uses])
    epsilon = 1e-10 # to avoid log(x) = -Inf
    loss = 0
    for i in range(int(m)):
        num = (tr.exp(-tr.sum((YY-XB[i])**2,2)/(2*sigma2))).mean(0) # f_{Y|B_i}
        den = (tr.exp(-tr.sum((YY-XX)**2,2)/(2*sigma2))).mean(0) # f_{Y}
        posterior = num/den
        loss = loss - (tr.log(posterior + epsilon)/np.log(2)).mean()
    return loss
          
          

def GMI_GH(X_tilde,idx):
    X = normalization(encoder(X_tilde))
    idx2 = np.zeros(M)
    for i in range(M):
        idx2[idx[i]] = i 
    X = X[idx2,:]
    Dmat = tr.zeros(M,M,channel_uses).to(Device)
    Dmat[:,:,0] = X[:,0].unsqueeze(1) -(X[:,0].unsqueeze(1)).t() #Calculate the distances between constellation points
    Dmat[:,:,1] = X[:,1].unsqueeze(1) -(X[:,1].unsqueeze(1)).t()
    
    Ik1 = np.zeros([int(M/2),int(m)]) #Find the pointers to the subconstellations
    Ik0 = np.zeros([int(M/2),int(m)])
    for kk in range(int(m)): 
        Ik1[:,kk] = np.where(labeling[:,kk] == 1)[0]
        Ik0[:,kk] = np.where(labeling[:,kk] == 0)[0]
    Ik1 = tr.tensor(Ik1, dtype = tr.int64).unsqueeze(1).to(Device)
    Ikden1 = Ik1 + M*tr.transpose(Ik1,0,1)
    Ik0 = tr.tensor(Ik0, dtype = tr.int64).unsqueeze(1).to(Device)
    Ikden0 = Ik0 + M*tr.transpose(Ik0,0,1)
    
    GH_xi = tr.tensor(GH['xi'], dtype  = tr.float32).to(Device)#Load in the Gauss-Hermite points
    GH_alpha = tr.tensor(GH['alpha'], dtype = tr.float32).to(Device)#Load in the Gauss-Hermite weigths
    
    Es = (X[:,0]**2 + X[:,1]**2).mean() #Calculate the signal energy
    EsN0lin = 10**(EsNo_dB/10)  #Turn the SNR value from dB to a linear value
    SigmaZ2 = (Es/(EsN0lin)) #Calculate the noise 
    
    num = tr.zeros((M,M)).to(Device)
    a = tr.zeros(M).to(Device)
    num0 = tr.zeros((M,m)).to(Device)
    num1 = tr.zeros((M,m)).to(Device)
    den0 = tr.zeros((M,m)).to(Device)
    den1 = tr.zeros((M,m)).to(Device)
    sum_0 = tr.zeros((int(M/2),m)).to(Device)
    sum_1 = tr.zeros((int(M/2),m)).to(Device)
    l_2 = tr.log(tr.tensor(2, dtype = tr.float32)).to(Device)
    GMI = 0
    for l1 in range(10): #Dimension 1
        for l2 in range(10): #Dimension 2
             num = tr.exp(-((Dmat[:,:,0]**2 + Dmat[:,:,1]**2) + 2*tr.sqrt(SigmaZ2)*(GH_xi[l1]*Dmat[:,:,0] - GH_xi[l2]*Dmat[:,:,1]))/SigmaZ2)
             a = num.sum(1)
             num0 = tr.squeeze(a.take(Ik0))
             den0 = tr.squeeze(num.take(Ikden0).sum(1))
             den1 = tr.squeeze(num.take(Ikden1).sum(1))
             num1 = tr.squeeze(a.take(Ik1))
             sum_0 = GH_alpha[l1]*GH_alpha[l2]*tr.log(num0/den0)/l_2 +sum_0
             sum_1 = GH_alpha[l1]*GH_alpha[l2]*tr.log(num1/den1)/l_2 +sum_1
    sum_0 = tr.sum(sum_0)
    sum_1 = tr.sum(sum_1)
    GMI = m-1/M/math.pi*(sum_0 + sum_1)
    loss = -GMI 
    return loss

def GMI_GH_4D(X_tilde,idx):
    X = normalization(encoder(X_tilde))
    idx2 = np.zeros(M)
    for i in range(M):
        idx2[idx[i]] = i 
    X = X[idx2,:]
    Dmat = tr.zeros(M,M,channel_uses).to(Device)
    Dmat[:,:,0] = X[:,0].unsqueeze(1) -(X[:,0].unsqueeze(1)).t() #Calculate the distances between constellation points
    Dmat[:,:,1] = X[:,1].unsqueeze(1) -(X[:,1].unsqueeze(1)).t()
    Dmat[:,:,2] = X[:,2].unsqueeze(1) -(X[:,2].unsqueeze(1)).t()
    Dmat[:,:,3] = X[:,3].unsqueeze(1) -(X[:,3].unsqueeze(1)).t()
    
    Ik1 = np.zeros([int(M/2),int(m)]) #Find the pointers to the subconstellations
    Ik0 = np.zeros([int(M/2),int(m)])
    for kk in range(int(m)): 
        Ik1[:,kk] = np.where(labeling[:,kk] == 1)[0]
        Ik0[:,kk] = np.where(labeling[:,kk] == 0)[0]
    Ik1 = tr.tensor(Ik1, dtype = tr.int64).unsqueeze(1).to(Device)
    Ikden1 = Ik1 + M*tr.transpose(Ik1,0,1).to(Device)
    Ik0 = tr.tensor(Ik0, dtype = tr.int64).unsqueeze(1).to(Device)
    Ikden0 = Ik0 + M*tr.transpose(Ik0,0,1).to(Device)
    
    GH_xi = tr.tensor(GH['xi'], dtype  = tr.float32).to(Device)#Load in the Gauss-Hermite points
    GH_alpha = tr.tensor(GH['alpha'], dtype = tr.float32).to(Device)#Load in the Gauss-Hermite weigths
    
    Es = (X[:,0]**2 + X[:,1]**2 + X[:,2]**2 + X[:,3]**2 ).mean() #Calculate the signal energy
    EsN0lin = 10**(EsNo_dB/10)  #Turn the SNR value from dB to a linear value
    SigmaZ2 = (Es/(EsN0lin)) #Calculate the noise 
    sum_0 = 0 #Initialize the sum 
    sum_1 = 0
    Dmatnorm = (Dmat[:,:,0]**2 + Dmat[:,:,1]**2 + Dmat[:,:,2]**2 + Dmat[:,:,3]**2)
    for l1 in range(10): #Dimension 1
        for l2 in range(10): #Dimension 2
            for l3 in range(10):
                for l4 in range(10):
                     num = tr.exp(-(Dmatnorm + tr.sqrt(2*SigmaZ2)*(GH_xi[l1]*Dmat[:,:,0] - GH_xi[l2]*Dmat[:,:,1] + GH_xi[l3]*Dmat[:,:,2] - GH_xi[l4]*Dmat[:,:,3]))/(0.5*SigmaZ2))
                     a = num.sum(1)                  
                     num0 = tr.squeeze(a.take(Ik0))
                     den0 = tr.squeeze(num.take(Ikden0).sum(1))
                     den1 = tr.squeeze(num.take(Ikden1).sum(1))
                     num1 = tr.squeeze(a.take(Ik1))
                     sum_0 = GH_alpha[l1]*GH_alpha[l2]*GH_alpha[l3]*GH_alpha[l4]*tr.log(num0/den0)/tr.log(tr.tensor(2, dtype = tr.float32))  + sum_0   
                     sum_1 = GH_alpha[l1]*GH_alpha[l2]*GH_alpha[l3]*GH_alpha[l4]*tr.log(num1/den1)/tr.log(tr.tensor(2, dtype = tr.float32))  + sum_1
    sum_0 = tr.sum(sum_0)
    sum_1 = tr.sum(sum_1)
    GMI = m-1/M/((math.pi)**2)*(sum_0 + sum_1)
    loss = -GMI 
    return loss


In [ ]:
### Training the model
start_time = time.time()
loss_history = np.zeros(epochs) # For saving the losses
Constellations = np.zeros((M,channel_uses,epochs)) #For saving the constellations
e = 0
encoder = tr.nn.Sequential()
encoder.add_module('last', tr.nn.Linear(M,channel_uses,bias = False))
encoder.to(Device)
optimizer = tr.optim.Adam(encoder.parameters(), learning_rate)
while e == 0:
    loss = sum(sum(((X_target - normalization(encoder(X_eye)))**2)))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if loss < 10**-8:
        e = 1
for i in range(1, epochs+1):
    if Estimation_type == 'GH':# and i > 500: #Uncomment this if you want the first several iterations to be calculated using MC, advised when doing 4D optimizations
        if channel_uses ==2:
            loss = GMI_GH(X_eye, idx_train)
        else:
            loss = GMI_GH_4D(X_eye, idx_train)
    else:
        loss = GMI(X_tilde, idx_train)
    Constellations[:,:,i-1] = normalization(encoder(X_eye)).detach().cpu().numpy()
    if i%100 == 0 or i ==1:
        print('iter ', i, ' loss', loss, 'time', time.time()-start_time)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_history[i-1] = loss.detach().cpu().numpy()
    save()

In [ ]:
#Plotting the final constellation with labeling
constellation = Constellations[:,:,-1]
x = constellation[:, 0] #Change from 0 to 2 for the second 2D for 4D constellations
y = constellation[:, 1] #Change from 1 to 3 for the second 2D for 4D constellations
max_axis = 1.8
fig = plt.figure(figsize=(m*3-3,3))

for i in range(int(m)):
	ax1 = plt.subplot(1, m, i+1)
	ax1.scatter(x[labeling[:,i] == 0], y[labeling[:,i] == 0], c='b', marker='.')
	ax1.scatter(x[labeling[:,i] == 1], y[labeling[:,i] == 1], c='r', marker='.')
	
	plt.xlabel('X')
	plt.ylabel('Y')
	plt.grid()
	#plt.axis('equal')
	plt.gca().set_aspect('equal', adjustable='box')
	plt.xlim(-max_axis, max_axis)
	plt.ylim(-max_axis, max_axis)
	ax1.set_title('bit position {}'.format(i))

plt.show()

In [ ]:
#Creating an animation of the evolution of the constellation, only works for 2D constellations
plt.rcParams['animation.embed_limit'] = 2**128
def init():
    line[0].set_data([],[])
    line[1].set_data([],[])
    return(line,)

def animate(i):
    x = Constellations[:,0,i]
    y = Constellations[:,1,i]
    line[0].set_data(x,y)
    line[1].set_data(np.linspace(0,i,i),loss_history[0:i])
    return(line,)


fig = plt.figure()
ax = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax.set_xlim((-1.8,1.8))
ax.set_ylim((-1.8,1.8))
ax2.set_xlim((-1,epochs))
ax2.set_ylim((-8.75,-8.6))
ax.set_aspect('equal')
ax2.set_xlabel('iterations')
ax2.set_ylabel('loss')
line1, = ax.plot([],[],'.',lw=2,)
line2, = ax2.plot([],[],lw=2,)
line1.set_color('b')
line2.set_color('b')
line = [line1, line2]
anim = animation.FuncAnimation(fig,animate,init_func = init, frames = epochs, interval = 10, blit = False)
anim.save('./Data/GMI/' + str(channel_uses) + 'D/' + str(M) + '/GMI_' + Estimation_type + '_' + str(channel_uses) + 'D_' + str(M) + '_' + str(EsNo_dB) + 'dB_'+ str(learning_rate)+'lr_' + Initial_Constellation + '_animation.mp4') 
HTML(anim.to_jshtml())